# Visium HD example

Example notebook for visualization of Visium HD data. 

This notebook is a miminal version of [this](../visium_hd/visium_hd.ipynb) notebook.

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
microns_per_pixel = 0.27376126715315274  # microns per pixel for visium HD

#### Fetch the example data

In [5]:
import os

import harpy as hp
import pooch
from harpy.datasets import get_registry
from spatialdata import read_zarr

BASE_DIR = "/Users/arne.defauw/VIB/DATA/vitessce_data/visium_hd_tutorial/"  # change the base directory

# fetch the data
registry = get_registry()
unzip_path = registry.fetch(
    "transcriptomics/visium_hd/mouse/visium_hd_mouse_small_intestine.zip",
    processor=pooch.Unzip(),
)

# path to a visium experiment
path = os.path.commonpath(unzip_path)

sdata = hp.io.visium_hd(
    path=path,
    bin_size=[16],
    dataset_id="Visium_HD_Mouse_Small_Intestine",
    bins_as_squares=True,
    output=os.path.join(BASE_DIR, "sdata_visium_hd_zarr2.zarr"),
)

sdata = read_zarr(os.path.join(BASE_DIR, "sdata_visium_hd_zarr2.zarr"))

/Users/arne.defauw/VIB/harpy_vitessce/.venv_harpy_vitessce_zarr2/lib/python3.12/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/Users/arne.defauw/VIB/harpy_vitessce/.venv_harpy_vitessce_zarr2/lib/python3.12/site-packages/anndata/_core/anndata.py:1825: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


INFO     The instance_key column in 'table.obs' ('table.obs[location_id]') will be relabeled to ensure a numeric   
         data type, with a continuous range and without including the value 0 (which is reserved for the           
         background). The new labels will be stored in a new column named 'relabeled_location_id'.                 


/Users/arne.defauw/VIB/harpy_vitessce/.venv_harpy_vitessce_zarr2/lib/python3.12/site-packages/spatialdata/_core/spatialdata.py:511: UserWarning: Converting `region_key: region` to categorical dtype.
  convert_region_column_to_categorical(table)


INFO     The Zarr backing store has been changed from None the new file path:                                      
         /Users/arne.defauw/VIB/DATA/vitessce_data/visium_hd_tutorial/sdata_visium_hd_zarr2.zarr                   


version mismatch: detected: RasterFormatV02, requested: FormatV04
/Users/arne.defauw/VIB/harpy_vitessce/.venv_harpy_vitessce_zarr2/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/arne.defauw/VIB/harpy_vitessce/.venv_harpy_vitessce_zarr2/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/arne.defauw/VIB/harpy_vitessce/.venv_harpy_vitessce_zarr2/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/arne.defauw/VIB/harpy_vitessce/.venv_harpy_vitessce_zarr2/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(co

#### Set the transformations.

In [6]:
from spatialdata.transformations import Scale, Identity
from spatialdata.transformations import set_transformation

transformations = {
    "micron": Scale(axes=("x", "y"), scale=[microns_per_pixel, microns_per_pixel]),
    "global": Identity(),  # Note that we need global coordinate sytem for ome ngff.
}

set_transformation(
    sdata["Visium_HD_Mouse_Small_Intestine_full_image"],
    transformation=transformations,
    set_all=True,
    write_to_sdata=sdata,
)

In [7]:
from spatialdata.models import TableModel

sdata["square_016um"].obsm["spatial_micron"] = (
    sdata["square_016um"].obsm["spatial"]
    * microns_per_pixel  # .obsm["spatial"] is in pixel coordinates -> need to convert to micron space
)

# back to zarr
sdata = hp.tb.add_table_layer(
    sdata,
    adata=sdata["square_016um"].copy(),
    region=sdata["square_016um"].uns[TableModel.ATTRS_KEY][TableModel.REGION_KEY],
    output_layer="square_016um",
    overwrite=True,
)

2026-02-16 16:00:06.589 | WARNING  | harpy.utils._io:_incremental_io_on_disk:53 - layer with name 'square_016um' already exists. Overwriting...


In [ ]:
sdata[
    "square_016um"
].X  # -> for optimal performance, verify that .X is a sparse csc matrix.

In [8]:
from harpy_vitessce.vitessce_config import visium_hd

from IPython.display import display, HTML

vc = visium_hd(
    sdata,
    img_layer="Visium_HD_Mouse_Small_Intestine_full_image",
    table_layer="square_016um",
    to_coordinate_system="micron",  # note that one could also choose to specify microns per pixels via parameter 'microns_per_pixel'
    spot_radius_size_micron=8,
    zoom=-3.2,
    visualize_as_rgb=True,
    spatial_key="spatial_micron",  # pass correct micron spatial key -> for visualization of spots, vitessce uses the spatial key in the AnnData
    qc_obs_feature_keys=None,
    cluster_key=None,
    embedding_key=None,
)

url = vc.web_app()
display(HTML(f'<a href="{url}" target="_blank">Open in Vitessce</a>'))

2026-02-16 16:00:09.855 | INFO     | harpy_vitessce.vitessce_config._seqbased_transcriptomics:visium_hd:431 - Both coordinate_transformations and microns_per_pixel is None.Fetching coordinate transformation from the SpatialData object.
